<a href="https://colab.research.google.com/github/golesuman/66-daysofdatachallenge/blob/main/Day52/simpleGanipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torchvision.datasets as datasets
import torch 
import matplotlib.pyplot as plt
import numpy as np
import torch.optim  as optim
from torch.utils.data import DataLoader,Dataset
import torchvision.transforms as transforms
import torch.nn as nn

In [2]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
transformer=transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize(mean=(0.5,),
                                                     std=(0.5,))
                                ])

train_data=datasets.MNIST(root='./data',train=True,transform=transformer,download=True)
train_loader=DataLoader(train_data,batch_size=128,shuffle=True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [3]:
# defining discriminator Class
class Discriminator(nn.Module):
  def __init__(self):
    super(Discriminator,self).__init__()
    self.model=nn.Sequential(nn.Linear(784,1024),
                             nn.LeakyReLU(0.2),
                             nn.Dropout(0.3),
                             nn.Linear(1024,512),
                             nn.LeakyReLU(0.2),
                             nn.Dropout(0.3),
                             nn.Linear(512,256),
                             nn.LeakyReLU(0.2),
                             nn.Dropout(0.3),
                             nn.Linear(256,1),
                             nn.Sigmoid()
                             )
    
  def forward(self,x):
    out=self.model(x)
    return out
from torchsummary import summary
discriminator=Discriminator().to(device)
print(discriminator)


Discriminator(
  (model): Sequential(
    (0): Linear(in_features=784, out_features=1024, bias=True)
    (1): LeakyReLU(negative_slope=0.2)
    (2): Dropout(p=0.3, inplace=False)
    (3): Linear(in_features=1024, out_features=512, bias=True)
    (4): LeakyReLU(negative_slope=0.2)
    (5): Dropout(p=0.3, inplace=False)
    (6): Linear(in_features=512, out_features=256, bias=True)
    (7): LeakyReLU(negative_slope=0.2)
    (8): Dropout(p=0.3, inplace=False)
    (9): Linear(in_features=256, out_features=1, bias=True)
    (10): Sigmoid()
  )
)


In [4]:
class Generator(nn.Module):
  def __init__(self):
    super(Generator,self).__init__()
    self.model=nn.Sequential(
        nn.Linear(100,256),
        nn.LeakyReLU(0.2),
        nn.Linear(256,512),
        nn.LeakyReLU(0.2),
        nn.Linear(512,1024),
        nn.LeakyReLU(0.2),
        nn.Linear(1024,784),
        nn.Tanh()
    )
  def forward(self,x):
    return self.model(x)
generator=Generator().to(device)

print(generator)


Generator(
  (model): Sequential(
    (0): Linear(in_features=100, out_features=256, bias=True)
    (1): LeakyReLU(negative_slope=0.2)
    (2): Linear(in_features=256, out_features=512, bias=True)
    (3): LeakyReLU(negative_slope=0.2)
    (4): Linear(in_features=512, out_features=1024, bias=True)
    (5): LeakyReLU(negative_slope=0.2)
    (6): Linear(in_features=1024, out_features=784, bias=True)
    (7): Tanh()
  )
)


In [5]:
! pip install -q torch_snippets
from torch_snippets import *
from torchvision.utils import make_grid

     |████████████████████████████████| 43 kB 701 kB/s 
     |████████████████████████████████| 213 kB 7.4 MB/s 
     |████████████████████████████████| 57 kB 5.0 MB/s 
     |████████████████████████████████| 10.1 MB 33.3 MB/s 
     |████████████████████████████████| 56 kB 4.4 MB/s 
     |████████████████████████████████| 51 kB 6.4 MB/s 


In [8]:
def noise(size):
  n=torch.randn(size,100)
  return n.to(device)

def discriminator_train(real_data,fake_data):
  d_optimizer.zero_grad()
  prediction_real=discriminator(real_data)
  error_real=loss(prediction_real,torch.ones(len(real_data),1).to(device))
  error_real.backward()
  prediction_fake=discriminator(fake_data)
  error_fake=loss(prediction_fake,torch.zeros(len(fake_data),1).to(device))
  error_fake.backward()
  d_optimizer.step()
  return error_real+error_fake

def generator_train(fake_data):
  g_optimizer.zero_grad()
  prediction=discriminator(fake_data)
  error=loss(prediction,torch.ones(len(real_data),1).to(device))
  error.backward()
  g_optimizer.step()
  return error
discriminator=Discriminator().to(device)
generator=Generator().to(device)
d_optimizer=optim.Adam(discriminator.parameters(),lr=0.0002)
g_optimizer=optim.Adam(generator.parameters(),lr=0.0002)

loss=nn.BCELoss()
num_epochs=200
log=Report(num_epochs)





In [ ]:
for epoch in range(num_epochs):
  N=len(train_loader)
  for i, (images,_) in enumerate(train_loader):
    real_data=images.view(len(images),-1).to(device)
    fake_data=generator(noise(len(real_data))).to(device)
    fake_data=fake_data.detach()
    d_loss=discriminator_train(real_data,fake_data)
    fake_data=generator(noise(len(real_data))).to(device)
    g_loss=generator_train(fake_data)
    log.record(epoch+(1+i)/N, d_loss=d_loss.item(),g_loss=g_loss.item(),\
               end='\r')
  log.report_avgs(epoch+1)
log.plot_epochs(['d_loss','g_loss'])

EPOCH: 1.000	d_loss: 0.612	g_loss: 4.530	(22.53s - 4483.08s remaining)
EPOCH: 2.000	d_loss: 0.003	g_loss: 9.314	(42.45s - 4202.57s remaining)
EPOCH: 3.000	d_loss: 0.010	g_loss: 12.891	(62.33s - 4092.83s remaining)
EPOCH: 3.045	d_loss: 0.000	g_loss: 13.944	(63.21s - 4088.81s remaining)